## Method 1: Using built-in `sorted` function.

In [4]:
class DebtSimplifier:
    def __init__(self):
        self.debts = []

    def add_debt(self, borrower, lender, amount):
        self.debts.append((borrower, lender, amount))
        self.net_balance = {}
        for borrower, lender, amount in self.debts:
            self.net_balance[borrower] = self.net_balance.get(borrower, 0) - amount
            self.net_balance[lender] = self.net_balance.get(lender, 0) + amount
    
    def simplify_debts(self):
        net_balance_copy = self.net_balance.copy()
        sorted_balances = sorted(net_balance_copy.items(), key=lambda x: x[1]) # Here we used sorted function
    
        debtors = 0
        creditors = len(sorted_balances) - 1
        transactions = []
    
        while debtors < creditors:
            debtor, debtor_balance = sorted_balances[debtors]
            creditor, creditor_balance = sorted_balances[creditors]
    
            if debtor_balance == 0:
                debtors += 1
                continue
            if creditor_balance == 0:
                creditors -= 1
                continue
    
            transfer_amount = min(-debtor_balance, creditor_balance)
            debtor_balance += transfer_amount
            creditor_balance -= transfer_amount
    
            transactions.append((debtor, creditor, transfer_amount))
    
            sorted_balances[debtors] = (debtor, debtor_balance)
            sorted_balances[creditors] = (creditor, creditor_balance)
    
            if debtor_balance == 0:
                debtors += 1
            if creditor_balance == 0:
                creditors -= 1
    
        return transactions
    
# Example usage:
simplifier = DebtSimplifier()
simplifier.add_debt("Alice", "Bob", 40)
simplifier.add_debt("Bob", "Charlie", 20)
simplifier.add_debt("Charlie", "David", 50)
simplifier.add_debt("Fred", "Bob", 10)
simplifier.add_debt("Fred", "Charlie", 30)
simplifier.add_debt("Fred", "David", 30)
simplifier.add_debt("Fred", "Gabe", 10)
simplifier.add_debt("Gabe", "Alice", 30)
simplifier.add_debt("Gabe", "Charlie", 10)
simplifier.add_debt("Gabe", "Fred", 20)


simplified_transactions = simplifier.simplify_debts()
print("The optimal transaction is")
for debtor, creditor, amount in simplified_transactions:
    print(f"{debtor} owes {creditor} ${amount}")


The optimal transaction is
Fred owes David $60
Gabe owes David $20
Gabe owes Bob $30
Alice owes Charlie $10


In [5]:
simplifier.net_balance.items()

dict_items([('Alice', -10), ('Bob', 30), ('Charlie', 10), ('David', 80), ('Fred', -60), ('Gabe', -50)])

The following code uses the built in 'sorted' function too, with the difference that it accepts multiple debtors

In [25]:
import pandas as pd

class Expense:
    def __init__(self, payer, debtors, amount):
        self.payer = payer
        self.debtors = debtors
        self.amount = amount

# This function calculates how much each debtor owes to the payer
def calculate_debts(expenses):
    balances = {}

    for expense in expenses:
        # The amount owed per debtor is the total amount divided by the number of debtors
        amount_per_debtor = expense.amount / len(expense.debtors)
        for debtor in expense.debtors:
            if debtor != expense.payer:  # Payer should not owe money to themselves
                # Update the debtor's balance
                balances[debtor] = balances.get(debtor, 0) - amount_per_debtor
                # Update the payer's balance
                balances[expense.payer] = balances.get(expense.payer, 0) + amount_per_debtor

    # Convert balances to a list of transactions
    transactions = [{'debtor': debtor, 'payer': payer, 'amount': -amount}
                    for debtor, amount in balances.items() if amount < 0
                    for payer, payer_amount in balances.items() if payer_amount > 0 and payer != debtor]

    # Sort transactions by amount in descending order
    sorted_transactions = sorted(transactions, key=lambda x: x['amount'], reverse=True)

    return sorted_transactions

# Example usage with hardcoded data
expenses = [
    Expense('elena', ['elena', 'minho', 'lino'], 1000.0),
    Expense('minho', ['elena', 'minho'], 500.0),
    Expense('minho', ['minho'], 500.0),
    Expense('elena', ['elena', 'minho'], 100.0),
    Expense('lino', ['elena', 'minho', 'lino'], 10.0)
]

debt_table = calculate_debts(expenses)

'''# to see another display of the table, uncomment from here.....'''
# # Display the output table
# for transaction in debt_table:  
# # following code says who owes how much to who, idk if we want it or not 
#     print(f"{transaction['debtor']} owes {transaction['payer']}: {transaction['amount']}") 

    
#     debt_table = calculate_debts(expenses)

# # Convert the list of transactions to a pandas DataFrame for tabular display
#     df = pd.DataFrame(debt_table)
#     #print(df)

# # Print the DataFrame as a table
#     print(df.to_string(index=False)) 
'''....to here'''

debt_table = calculate_debts(expenses)

# Convert the list of transactions to a pandas DataFrame for tabular display
df = pd.DataFrame(debt_table)

# Ensure only one output table by commenting out or removing any duplicate print statements
# Print the DataFrame as a table
print(df.to_string(index=False))

debtor payer     amount
  lino elena 326.666667
 minho elena 136.666667


## Method 2: Using merge sort algorithm

In [6]:
class DebtSimplifier:
    def __init__(self):
        self.debts = []

    def add_debt(self, borrower, lender, amount):
        self.debts.append((borrower, lender, amount))
        self.net_balance = {}
        for borrower, lender, amount in self.debts:
            self.net_balance[borrower] = self.net_balance.get(borrower, 0) - amount
            self.net_balance[lender] = self.net_balance.get(lender, 0) + amount
    def merge_sort(self,arr, compare_func=None):
        arr_temp = list(arr)
        n = len(arr_temp)    
    
        if n > 1: 
            mid = n // 2
            arr_temp_left = arr_temp[:mid] 
            arr_temp_right = arr_temp[mid:]
      
            arr_temp_left = self.merge_sort(arr_temp_left, compare_func)
            arr_temp_right = self.merge_sort(arr_temp_right, compare_func)
              
            i = j = k = 0
            n_left, n_right = len(arr_temp_left), len(arr_temp_right)
              
            while i < n_left and j < n_right: 
                if compare_func(arr_temp_left[i], arr_temp_right[j]):
                    arr_temp[k] = arr_temp_left[i] 
                    i += 1
                else: 
                    arr_temp[k] = arr_temp_right[j] 
                    j += 1
                k += 1
              
            while i < n_left: 
                arr_temp[k] = arr_temp_left[i] 
                i += 1
                k += 1
     
            while j < n_right: 
                arr_temp[k] = arr_temp_right[j] 
                j += 1
                k += 1
                
        return arr_temp

    def merge_sort_dict_by_value(self,dictionary):

        def compare_dict_items(item1, item2):

            return item1[1] < item2[1]
    
        items = list(dictionary.items())
        sorted_items = self.merge_sort(items, compare_func=compare_dict_items)
        return dict(sorted_items)
    
    def simplify_debts(self):
        net_balance_copy = self.net_balance.copy()
        sorted_balances = list(self.merge_sort_dict_by_value(net_balance_copy).items())
        debtors = 0
        creditors = len(sorted_balances) - 1
        transactions = []
        

        while debtors < creditors:
            debtor, debtor_balance = sorted_balances[debtors]
            creditor, creditor_balance = sorted_balances[creditors]
    
            if debtor_balance == 0:
                debtors += 1
                continue
            if creditor_balance == 0:
                creditors -= 1
                continue
    
            transfer_amount = min(-debtor_balance, creditor_balance)
            debtor_balance += transfer_amount
            creditor_balance -= transfer_amount
    
            transactions.append((debtor, creditor, transfer_amount))
    
            sorted_balances[debtors] = (debtor, debtor_balance)
            sorted_balances[creditors] = (creditor, creditor_balance)
    
            if debtor_balance == 0:
                debtors += 1
            if creditor_balance == 0:
                creditors -= 1
    
        return transactions
            
        
        
simplifier = DebtSimplifier()
simplifier.add_debt("Alice", "Bob", 40)
simplifier.add_debt("Bob", "Charlie", 20)
simplifier.add_debt("Charlie", "David", 50)
simplifier.add_debt("Fred", "Bob", 10)
simplifier.add_debt("Fred", "Charlie", 30)
simplifier.add_debt("Fred", "David", 30)
simplifier.add_debt("Fred", "Gabe", 10)
simplifier.add_debt("Gabe", "Alice", 30)
simplifier.add_debt("Gabe", "Charlie", 10)
simplifier.add_debt("Gabe", "Fred", 20)

simplified_transactions = simplifier.simplify_debts()
print("The optimal transaction is")
for debtor, creditor, amount in simplified_transactions:
    print(f"{debtor} owes {creditor} ${amount}")
        
        
        
        

The optimal transaction is
Fred owes David $60
Gabe owes David $20
Gabe owes Bob $30
Alice owes Charlie $10


Same but accepting multiple inputs 

In [26]:
import pandas as pd

class Expense:
    def __init__(self, payer, debtors, amount):
        self.payer = payer
        self.debtors = debtors.split(', ')  # Splitting the string of debtors into a list
        self.amount = amount

def calculate_debts(expenses):
    balances = {}

    for expense in expenses:
        amount_per_debtor = expense.amount / len(expense.debtors)
        for debtor in expense.debtors:
            if debtor != expense.payer:  # Skip if the debtor is also the payer
                balances[debtor] = balances.get(debtor, 0) - amount_per_debtor
                balances[expense.payer] = balances.get(expense.payer, 0) + amount_per_debtor

    transactions = [{'debtor': debtor, 'payer': payer, 'amount': -amount}
                    for debtor, amount in balances.items() if amount < 0
                    for payer, payer_amount in balances.items() if payer_amount > 0 and payer != debtor]

    sorted_transactions = merge_sort(transactions, key=lambda x: x['amount'])

    return sorted_transactions

def merge_sort(arr, key=lambda x: x):
    if len(arr) > 1:
        mid = len(arr) // 2
        L = arr[:mid]
        R = arr[mid:]

        merge_sort(L, key=key)
        merge_sort(R, key=key)

        i = j = k = 0

        while i < len(L) and j < len(R):
            if key(L[i]) > key(R[j]):
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1

        while i < len(L):
            arr[k] = L[i]
            i += 1
            k += 1

        while j < len(R):
            arr[k] = R[j]
            j += 1
            k += 1

    return arr

# Example usage with hardcoded data
expenses = [
    Expense('elena', 'elena, minho, lino', 1000.0),
    Expense('minho', 'elena, minho', 500.0),
    Expense('minho', 'minho', 500.0),
    Expense('elena', 'elena, minho', 100.0),
    Expense('lino', 'elena, minho, lino', 10.0)
]

debt_table = calculate_debts(expenses)

# Convert the list of transactions to a pandas DataFrame for tabular display
df = pd.DataFrame(debt_table)

print(df.to_string(index=False))


debtor payer     amount
  lino elena 326.666667
 minho elena 136.666667
